# Demo de Deep Autoencoder (DAE) para procesar las imágenes de algunos TIPOS de ANIMALES
Basado en: 

https://blog.keras.io/building-autoencoders-in-keras.html


https://towardsdatascience.com/deep-autoencoders-using-tensorflow-c68f075fd1a3

1) Importar librerías:

In [ ]:
#@title Librerías a usar
import keras
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import plot_model

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

from PIL import Image

print("\nLibrerías importadas")

2) Definir la configuración del modelo DAE:

In [ ]:
## selección de los parámetros 

# 
#@markdown ### Parámetros de imágenes:
imagen_largo_ancho = 20 #@param {type:"integer"}
imagen_color = True #@param {type:"boolean"}

#@markdown ### Parámetros de la red:
rna_cant_neuronas_capas_ocultas = '392, 196, 84, 56, 32' #@param {type:"string"}
rna_cant_neuronas_features = 12 #@param {type:"integer"}
rna_cant_epocas_entrenamiento = 300 #@param {type:"integer"}


## aplicación de los parámetros elegidos

# tamaño de las imágenes
if imagen_largo_ancho<=10:
  imagen_largo_ancho = 10
IMAGE_SHAPE = (imagen_largo_ancho, imagen_largo_ancho, (3 if imagen_color else 1))

# cantidad de neuronas ocultas para features (datos comprimidos o codings)
num_features = (2 if rna_cant_neuronas_features<2 else rna_cant_neuronas_features)

# define tamaño de datos de entrada 
num_inputs = IMAGE_SHAPE[0] * IMAGE_SHAPE[1] * IMAGE_SHAPE[2]
num_outputs = num_inputs

# cantidad de neuronas ocultas 
##hidden_layers = [ num_inputs//5, num_inputs//20, num_inputs//100 ]
dae_layers = []
for val in rna_cant_neuronas_capas_ocultas.split(','):
  dae_layers.append( int(val) )
      
#  agrega la capa de features a las capas
dae_layers.append( num_features ) 

# cantidad de neuronas ocultas para la parte Decoder 
#   (usa la la lista de Encoder inversa)
for eachEncLayer in dae_layers[0:len(dae_layers)-1][::-1]:
  dae_layers.append( eachEncLayer )

# cantidad de épocas del entrenamiento
cantEpocas = (100 if rna_cant_epocas_entrenamiento<1 else rna_cant_epocas_entrenamiento)

print("Configuración del DAE definida: [", num_inputs, dae_layers, num_outputs, "] ")

3) Montar el Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# directorio local en Google Drive
path = 'gdrive/My Drive/IA/demo ANIMALES/imagenes' #@param {type:"string"}
path_entrenamiento = '/train'  #@param {type:"string"}
path_prueba = '/test'  #@param {type:"string"}

imagPath_train = path + path_entrenamiento
imagPath_test = path + path_prueba

4) Cargar imágenes para entrenar el modelo DAE:

In [ ]:
#@title Cargar Imágenes

# define función para cargar las imágenes
def cargarImagenes(imagPath):
  classes_ori = [] 
  images_ori = []
  esDA_ori = []

  all_dirs = os.listdir( imagPath )
  for each_dir in all_dirs:

      auxiPath = imagPath + '/' + each_dir 
      imagFN  = os.listdir( auxiPath )
      for each_imagFN in imagFN:

            esImagDA = (each_imagFN[:2] == 'da')
            
            # abre la imagen
            imag = Image.open(auxiPath + "/" + each_imagFN)
            
            # ajusta el tamaño
            if IMAGE_SHAPE[2]==1:              
              tipoImage = 'L'
            else:                
              tipoImage = 'RGB'
            imag = imag.convert(tipoImage)
            imag = imag.resize((IMAGE_SHAPE[0], IMAGE_SHAPE[1]), Image.ANTIALIAS)          
            
            # transforma a un vector de nros
            arImag = np.array(imag)
            
            # agrega a los vectores
            classes_ori.append( each_dir )
            images_ori.append( arImag )
            esDA_ori.append( esImagDA )

  return classes_ori, images_ori, esDA_ori, tipoImage

# carga las imagenes de entrenamiento
classes_train, images_train, esDAimag_train, tipoImage_train = cargarImagenes(imagPath_train)
print("> Para Entrenamiento: ")
print("- Clases cargadas: ", len(classes_train))
print("- Imágenes cargadas: ", len(classes_train))

if len(classes_train)>0:
  print("- Ejemplo ", classes_train[0], " ", images_train[0].shape, ": ")
  display( Image.fromarray(images_train[0], tipoImage_train) )

# carga las imagenes de prueba
classes_test, images_test, esDAimag_test, tipoImage_test = cargarImagenes(imagPath_test)
print("\n\n> Para Prueba: ")
print("- Clases cargadas: ", len(classes_test))
print("- Imágenes cargadas: ", len(images_test))

if len(classes_test)>0:
  print("- Ejemplo ", classes_test[0], " ", images_test[0].shape, ": ")
  display( Image.fromarray(images_test[0], tipoImage_test) )

In [ ]:
#@title Preparar imágenes

# define función auxiliar para mostrar imágenes preparadas
def plot_image(imag):
  if IMAGE_SHAPE[2]==1:
    plt.imshow((imag*255).reshape(IMAGE_SHAPE[0], IMAGE_SHAPE[1]).astype(np.uint8))
    plt.gray()
  else:
    plt.imshow((imag*255).reshape(IMAGE_SHAPE).astype(np.uint8))
  plt.axis("off")  

# define función auxiliar para preparar la lista de imágenes a procesar
def prepare_imageList(imagList):    
  auxiAr = np.array(imagList).astype('float32') / 255.
  auxiAr = auxiAr.reshape((len(auxiAr), num_inputs))  
  return np.array(auxiAr)

# define vector auxiliar de datos de entrada para usar en el entrenamiento y prueba
x_train = prepare_imageList(images_train)
x_test = prepare_imageList(images_test)

print("> Para Entrenamiento: ")
print(" - x_train (cant ejemplos, datos entrada): ", x_train.shape)
print("\n\n> Para Prueba: ")
print(" - x_test (cant ejemplos, datos entrada): ", x_test.shape)
if len(x_train)>0:
  plot_image(x_train[0])

5) Creación del modelo DAE:

In [ ]:
#@title Establecer el modelo

# define la arquitectura de capas del Deep Autoencoder
# teniendo en cuenta la definición dada anteriomente
input_img_Lay = Input(shape=(num_inputs,), name='input_img') # capa de entrada
eachLay = input_img_Lay
auxName = 'enc_'
auxId = 1 
for num_hid in dae_layers:  

    # define el nombre de la capa oculta
    if num_features==num_hid:
        auxlayerName = 'features'
        auxName = 'dec_'
        auxId = auxId - 1
    else:
        auxlayerName = auxName+str(auxId)
        if auxName == 'enc_':
          auxId = auxId + 1
        else:
          auxId = auxId - 1

    # agrega la capa oculta
    eachLay = Dense(num_hid, activation='relu', name=auxlayerName)(eachLay) # capas ocultas

    if num_features==num_hid:
      features_Lay = eachLay

output_img_Lay = Dense(num_outputs, activation=None, name='output_img')(eachLay) # capa de salida

# genera el modelo Deep Autoencoder
DAEmodel = Model(input_img_Lay, output_img_Lay, name='DAE')
#DAEmodel.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
DAEmodel.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

print("Modelo DAE creado con ", len(DAEmodel.layers), " capas:")
DAEmodel.summary()
print("\n")
plot_model(DAEmodel, show_layer_names=True, show_shapes=True)

5) Entrenar el modelo DAE:

In [ ]:
#@title Entrenar

# lleva a cabo el entrenamiento
# usando los mismos datos como entrada y salida
DAEmodel.fit(x_train, x_train,
                epochs = cantEpocas,
                batch_size = 15)

7) Evaluar el modelo DAE entrenado solicitando que reconstruya las imágenes ingresadas:

In [ ]:
#@title Evaluar el modelo con las imágenes de entrenamiento

# función auxiliar para probar el modelo entrenado en detalle
def probarModelo(x, cl, esDAimag):

  # evalua al modelo 
  resEval = DAEmodel.evaluate(x, x)
  print("\n>Evaluación del Modelo: ")
  print("    - Error: ", resEval[0])
  print("    - Exactitud: ", resEval[1]*100)
  print("\n")

  # procesa las imágenes con el modelo 
  reconstr_imgs = DAEmodel.predict(x)

  # muestra las 15 primeras imágenes 
  print("\n>Resultados: ")
  for i in range(len(x)):
    # no muestra las generadas por DA
    if not esDAimag[i]:
          # prepara para mostrar
          fig = plt.figure()
          fig.suptitle(cl[i])

          # muestra la real
          ax1 = fig.add_subplot(121)
          plot_image(x[i])

          # muestra la generada por el modelo
          ax2 = fig.add_subplot(122)
          plot_image(reconstr_imgs[i])

          plt.tight_layout()
          fig = plt.gcf()


# prueba con los datos de entrenamiento
print("*** Resultados con datos de Entrenamiento: ")
probarModelo(x_train, classes_train, esDAimag_train)

9) Probar el modelo DAE entrenado con otras imágenes:

In [ ]:
#@title Evaluar el modelo con las imágenes de prueba

# prueba con los datos de prueba
print("*** Resultados con datos de Prueba: ")
probarModelo(x_test, classes_test, esDAimag_test)

8) A partir del modelo DAE entrenado, generar dos sub-modelos Encoder y Decoder:


*   Generar y usar el modelo Encoder para 'clusterizar' las imágenes de entrenamiento:


In [ ]:
#@title Generar el sub-modelo Encoder para Clustering
## (desde input hasta features)

# reutiliza las capas entrenadas del modelo DAE original
clust_input_Lay = input_img_Lay  # capa de entrada
clust_output_Lay =  features_Lay  # capa de salida

# genera el modelo
CLUSTmodel = Model(input_img_Lay, features_Lay, name='Encoder/Clustering')

print("> Modelo Encoder: ")
CLUSTmodel.summary()
plot_model(CLUSTmodel, show_layer_names=True, show_shapes=True)

In [ ]:
#@title Generar Clustering

# función auxiliar para generar un gráfico 
# con los valores codificados 
# usando PCA para simplificarlos en 2 ejes
def genera_grafico_pca(datos, clases, titulo):
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(datos)
    principalDf = pd.DataFrame(data = principalComponents,
                columns = ['pca_1', 'pca_2'])
    finalDf = pd.concat([principalDf, 
                        pd.DataFrame(clases, columns = ['target'])], 
                        axis = 1)

    fig = plt.figure(figsize = (10,10))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('PCA1', fontsize = 15)
    ax.set_ylabel('PCA2', fontsize = 15)
    ax.set_title(titulo, fontsize = 20)
    for target in set(clases):
        indicesToKeep = finalDf['target'] == target
        ax.scatter(finalDf.loc[indicesToKeep, 'pca_2'],
                  finalDf.loc[indicesToKeep, 'pca_1'],
                  s = 50)
    ax.legend(set(clases))
    ax.grid()

# procesa las imágenes para recibir el valor codificado de cada una
x_train_encoded = CLUSTmodel.predict(x_train)

# muestra el gráfico con imágenes originales
genera_grafico_pca(x_train, classes_train, "Representación de Imágenes Originales")

# muestra estadísticas de los datos codificados
minArClust = np.empty(num_features)
minArClust.fill(9999.99)
maxArClust = np.empty(num_features)
maxArClust.fill(-9999.99)
sumArClust = np.zeros(num_features)
for val in x_train_encoded:
  for i in range(num_features):
      sumArClust[i] = sumArClust[i]+val[i]
      if val[i]<minArClust[i]: 
          minArClust[i] = val[i]
      if val[i]>maxArClust[i]: 
          maxArClust[i] = val[i]
print("\n\n> Estadísticas de Clutering de Imágenes codificado en ", num_features," valores: ")
print("- Mínimos:   ", minArClust)
print("- Máximos:   ", maxArClust)
print("- Totales:   ", sumArClust)
print("- Promedios: ", sumArClust/len(x_train_encoded))
print("\n\n")

# muestra el gráfico codificado
genera_grafico_pca(x_train_encoded, classes_train, "Representación de Clustering de Imágenes")

*   Generar y usar el modelo Decoder para generar nuevas imágenes similares a las entrenadas:

In [ ]:
#@title Generar el sub-modelo Decoder para Generator
## (desde features hasta output)

# genera una copia del modelo DAE original para evitar romperlo
auxiCloneModel = keras.models.model_from_json(DAEmodel.to_json())
#auxiCloneModel.summary()

# genera la nueva estructura del Generator
input_gen = Input(shape=(num_features,), name='input_gen') # nueva capa de entrada
auxLay_gen = input_gen
for pos in range(len(DAEmodel.layers)):

  # obtiene el nombre de la capa actual
  auxName = DAEmodel.layers[pos].name  
  
  # sólo considera las capas luego de features (decoder y output)
  # para copiar los pesos del DAE original y actualizar la estrcutura
  if auxName.startswith('dec_') or auxName=='output_img':
    auxiCloneModel.layers[pos].set_weights(DAEmodel.layers[pos].get_weights()) 
    auxLay_gen = auxiCloneModel.layers[pos](auxLay_gen) 

# crea el nuevo modelo Generator
GENmodel = Model(input_gen, auxLay_gen, name = 'Decoder/Generator')

print("> Modelo Decoder: ")
GENmodel.summary()
plot_model(GENmodel, show_layer_names=True, show_shapes=True)

In [ ]:
#@title Ejecutar el Generator
#  usando valores definidos al azar como datos de entrada
cantImagenGenerar = 3
consideraEstadClust = True

# genera los datos de entrada
# (como la codificación tiene varias posiciones con ceros 
# se considera que sólo se ponen al azar entre 10% y el 70% de los valores, el resto queda en cero,
# --sino se podría hacer con " np.random.rand(cantImagenGenerar, num_features) "-- )
arX = []
if consideraEstadClust:
    # si están definidas las estadísticas de clustering,
    # genera los valores considerandolas
    print("Genera usando estadísticas de Clustering")
    for i in range(cantImagenGenerar):
        X = np.zeros(num_features)
        for pos in range(num_features):
            if sumArClust[pos]>0:
                X[pos] = np.random.uniform(minArClust[pos],maxArClust[pos])
        arX.append( X )
else:
      # si no están definidas las estadísticas de clustering,
    # genera los valores totalmente al azar
    print("Genera usando valores al azar")
    minRnd = num_features*10//100
    maxRnd = num_features*70//100
    for i in range(cantImagenGenerar):
        X = np.zeros(num_features)
        for j in range(np.random.randint(low=minRnd, high=maxRnd)):
            pos = np.random.randint(low=0, high=num_features-1)
            X[pos] = np.random.normal()
        arX.append( X )


# ejecuta el modelo Generator
imagOut = GENmodel.predict( np.array(arX).reshape((len(arX), num_features)) )  
 
# muestra las imágenes generadas
print("\n> Resultados: ")
for i in range(len(arX)):

    fig = plt.figure()

    # muestra los datos
    ax1 = fig.add_subplot(121)
    datosMostrar = arX[i].reshape(num_features, 1) 
    ax1.table(cellText=datosMostrar, loc='center')   
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)  

    #  muestra reconstrucción
    ax2 = fig.add_subplot(122)
    plot_image(imagOut[i])  

    plt.tight_layout()
    fig = plt.gcf()


*   Combinar los resultados de los modelos Clustering y Generator para mostrar que funcionan juntos como el DAE original:

In [ ]:
#@title Prueba el modelo Generator usando como entrada los datos de Clustering
pruebaClust = CLUSTmodel.predict( x_train )
pruebaClust_out = GENmodel.predict(  np.array(pruebaClust).reshape((len(pruebaClust), num_features)) )  
 
# muestra las imágenes generadas
print("\n> Resultados (valores de clustering, imagen recounstrida(grande) y original(chica): ")
for i in range(len(x_train_encoded)):

    # no muestra las generadas por DA
    if not esDAimag_train[i]:

        fig = plt.figure()

        # muestra los datos
        ax1 = fig.add_subplot(121)
        ax1.table(cellText=pruebaClust[i].reshape(num_features, 1), loc='center')   
        ax1.get_xaxis().set_visible(False)
        ax1.get_yaxis().set_visible(False)  

        #  muestra reconstrucción
        ax2 = fig.add_subplot(122)
        plot_image(pruebaClust_out[i])  

        # muestra imagen original
        ax3 = fig.add_subplot(332)
        plot_image(x_train[i])  

        fig = plt.gcf()